In [28]:
# Load packages
import numpy as np
import pandas as pd
from datetime import datetime
import datetime as dt
from twitterscraper import query_tweets

# 1. Querying Tweets

In [34]:
# Set up date range for the extraction
begindate = dt.date(2019,1,1)
enddate = dt.date.today() - dt.timedelta(days=1)

# Querying tweets using twitterscraper's query_tweets
# Query_tweets function:
'''
query_tweets(query = args.query, limit = args.limit,
                              begindate = args.begindate, enddate = args.enddate,
                              poolsize = args.poolsize, lang = args.lang)
'''

list_of_tweets = query_tweets("@bmo", limit = None, begindate = begindate, enddate = enddate, lang='en')
list_of_encoded_tweets = [] # create empty list to save multiple tweets which is type of dictionary

for tweets in list_of_tweets:
    tweets.timestamp = dt.datetime.strftime(tweets.timestamp, '%Y-%m-%d %H:%M:%S')
    list_of_encoded_tweets.append(vars(tweets))

print('--Querying Ended--')

INFO: queries: ['@bmo since:2019-01-01 until:2019-01-13', '@bmo since:2019-01-13 until:2019-01-26', '@bmo since:2019-01-26 until:2019-02-07', '@bmo since:2019-02-07 until:2019-02-20', '@bmo since:2019-02-20 until:2019-03-05', '@bmo since:2019-03-05 until:2019-03-17', '@bmo since:2019-03-17 until:2019-03-30', '@bmo since:2019-03-30 until:2019-04-12', '@bmo since:2019-04-12 until:2019-04-24', '@bmo since:2019-04-24 until:2019-05-07', '@bmo since:2019-05-07 until:2019-05-20', '@bmo since:2019-05-20 until:2019-06-01', '@bmo since:2019-06-01 until:2019-06-14', '@bmo since:2019-06-14 until:2019-06-27', '@bmo since:2019-06-27 until:2019-07-09', '@bmo since:2019-07-09 until:2019-07-22', '@bmo since:2019-07-22 until:2019-08-04', '@bmo since:2019-08-04 until:2019-08-16', '@bmo since:2019-08-16 until:2019-08-29', '@bmo since:2019-08-29 until:2019-09-11']
INFO: Got 500 tweets (500 new).
INFO: Got 962 tweets (462 new).
INFO: Got 1464 tweets (502 new).
INFO: Got 2026 tweets (562 new).
INFO: Got 2583

--Querying Ended--


In [49]:
# save output as df 

list_tweets = [list(elem.values()) for elem in list_of_encoded_tweets]
list_columns = list(list_of_encoded_tweets[0].keys())
df_bmo_tweets = pd.DataFrame(list_tweets, columns = list_columns)

# 2. Data Preprocessing
## 2.1 Consolidate Dataset

In [50]:
# Load previously saved CIBC tweets and app reviews
#df_tweets = pd.read_csv(r'bmo_tweets.csv',encoding='iso-8859-1')
print(df_bmo_tweets.head())

         username        fullname              user_id             tweet_id  \
0             BMO             BMO            222249603  1157761988640067586   
1   eleanorbeaton  Eleanor Beaton            227334930  1157751756039868416   
2  CalgaryBluesky  CalgaryBluesky  1154477958574137344  1157750150183264256   
3        karun_f7  Karun Mittal 👻            542400049  1157715685965008896   
4             BMO             BMO            222249603  1157694145319899136   

                                    tweet_url            timestamp  \
0             /BMO/status/1157761988640067586  2019-08-03 21:15:42   
1   /eleanorbeaton/status/1157751756039868416  2019-08-03 20:35:03   
2  /CalgaryBluesky/status/1157750150183264256  2019-08-03 20:28:40   
3        /karun_f7/status/1157715685965008896  2019-08-03 18:11:43   
4             /BMO/status/1157694145319899136  2019-08-03 16:46:07   

   timestamp_epochs  replies  retweets  likes  is_retweet retweeter_username  \
0        1564866942     

In [55]:
# rename the columns
df_bmo_tweets=df_bmo_tweets.rename(index=str, columns={'timestamp':'Date','fullname':'User','likes':'Rating/Likes','text':'Review'})

In [57]:
# Keep the timestamp and text columns
df_tweets_cleaned = df_bmo_tweets[['Date','User','Rating/Likes','Review']]
print(df_tweets_cleaned.head(10))
print('\nTotal Number of Original Tweets: {}\n'.format(len(df_tweets_cleaned)))

                  Date            User  Rating/Likes  \
0  2019-08-03 21:15:42             BMO             0   
1  2019-08-03 20:35:03  Eleanor Beaton             1   
2  2019-08-03 20:28:40  CalgaryBluesky             0   
3  2019-08-03 18:11:43  Karun Mittal 👻             2   
4  2019-08-03 16:46:07             BMO             0   
5  2019-08-03 16:35:04     Master Yoda             0   
6  2019-08-03 16:17:41      DaveMurYYC             0   
7  2019-08-03 16:15:54             BMO             0   
8  2019-08-03 15:53:58      DaveMurYYC             0   
9  2019-08-03 15:18:34       Clearbanc             0   

                                              Review  
0  Thank you Eleanor! It's such an honor for us. ^RJ  
1  On this EP  of  #FFL @_jeneroux offers tips fo...  
2  I'm not always in a rush or that impatient. Th...  
3  Iron ore prices are down sharply in China, par...  
4  You would need to visit a BMO ATM with your te...  
5  I don’t have the temp pin with me currently. I... 

In [58]:
# Convert them to lowercase
df_tweets_cleaned['Review'] = df_tweets_cleaned['Review'].str.lower()
print(df_tweets_cleaned.head(10))

                  Date            User  Rating/Likes  \
0  2019-08-03 21:15:42             BMO             0   
1  2019-08-03 20:35:03  Eleanor Beaton             1   
2  2019-08-03 20:28:40  CalgaryBluesky             0   
3  2019-08-03 18:11:43  Karun Mittal 👻             2   
4  2019-08-03 16:46:07             BMO             0   
5  2019-08-03 16:35:04     Master Yoda             0   
6  2019-08-03 16:17:41      DaveMurYYC             0   
7  2019-08-03 16:15:54             BMO             0   
8  2019-08-03 15:53:58      DaveMurYYC             0   
9  2019-08-03 15:18:34       Clearbanc             0   

                                              Review  
0  thank you eleanor! it's such an honor for us. ^rj  
1  on this ep  of  #ffl @_jeneroux offers tips fo...  
2  i'm not always in a rush or that impatient. th...  
3  iron ore prices are down sharply in china, par...  
4  you would need to visit a bmo atm with your te...  
5  i don’t have the temp pin with me currently. i... 

C:\Users\rio.w\AppData\Local\Continuum\anaconda3\envs\BR\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [60]:
# Remove duplicates
df_tweets_cleaned = df_tweets_cleaned.drop_duplicates(subset=['Review'])
print(df_tweets_cleaned.head(20))
print('\nTotal Number of De-duplicated Tweets: {}\n'.format(len(df_tweets_cleaned)))

                   Date              User  Rating/Likes  \
0   2019-08-03 21:15:42               BMO             0   
1   2019-08-03 20:35:03    Eleanor Beaton             1   
2   2019-08-03 20:28:40    CalgaryBluesky             0   
3   2019-08-03 18:11:43    Karun Mittal 👻             2   
4   2019-08-03 16:46:07               BMO             0   
5   2019-08-03 16:35:04       Master Yoda             0   
6   2019-08-03 16:17:41        DaveMurYYC             0   
7   2019-08-03 16:15:54               BMO             0   
8   2019-08-03 15:53:58        DaveMurYYC             0   
9   2019-08-03 15:18:34         Clearbanc             0   
10  2019-08-03 15:06:33               BMO             0   
11  2019-08-03 15:03:52               BMO             1   
12  2019-08-03 15:03:33       Master Yoda             0   
13  2019-08-03 14:56:09         Daniela V             2   
14  2019-08-03 13:29:17               BMO             0   
15  2019-08-03 13:17:50  Sarah Jane Wolch             0 

In [61]:
# Keep tweets that contain 'bmo'
df_tweets_cleaned = df_tweets_cleaned[df_tweets_cleaned['Review'].str.contains('bmo')]
print('\nTotal Number of De-duplicated Tweets that contain \'bmo\': {}\n'.format(len(df_tweets_cleaned)))


Total Number of De-duplicated Tweets that contain 'bmo': 8569



In [63]:
# Format the timestamp to date
df_tweets_cleaned['Date'] = [datetime.strptime(d[:10], '%Y-%m-%d').date() for d in df_tweets_cleaned['Date']]

In [64]:
df_tweets_cleaned.head(n=10)

,Date,User,Rating/Likes,Review
1,2019-08-03,Eleanor Beaton,1,on this ep of #ffl @_jeneroux offers tips fo...
2,2019-08-03,CalgaryBluesky,0,i'm not always in a rush or that impatient. th...
3,2019-08-03,Karun Mittal 👻,2,"iron ore prices are down sharply in china, par..."
4,2019-08-03,BMO,0,you would need to visit a bmo atm with your te...
8,2019-08-03,DaveMurYYC,0,i was in a online chat with @bmo agent and aft...
12,2019-08-03,Master Yoda,0,@bmo how do i set the pin to my credit card th...
13,2019-08-03,Daniela V,2,ok the staff at @bmo on 4th and national rock!...
15,2019-08-03,Sarah Jane Wolch,0,@bmo my 87yr grandmother went to cash travelle...
18,2019-08-03,Justin Case,0,@bmo so frustrating! doesnt matter when a pers...
19,2019-08-03,BMO,0,hello cameron! thank you for your feedback. i ...


In [66]:
# dropping all users that have 'BMO' in their username as we are interested only in clients and not bank's responses.
df_tweets_cleaned = df_tweets_cleaned[~df_tweets_cleaned.User.str.contains('BMO', na = False)]
print('\nTotal Number after dropping BMO reply: {}\n'.format(len(df_tweets_cleaned)))


Total Number after dropping BMO reply: 7785



In [67]:
# check NaN
df_tweets_cleaned.isna().sum()

Date            0
User            0
Rating/Likes    0
Review          0
dtype: int64

In [69]:
#save as csv
df_tweets_cleaned.to_csv(r'bmo_consolidated.csv', index=False)